[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/choROPeNt/gzlb/blob/main/notebooks/GZLB-UE6.4.ipynb
)

# GZLB
## UE6: Anisotropes Werkstoffverhalten, Verallgemeinertes Hooke'sches Gesetz, Festigkeitshypothesen



### Aufgabe 6.4: Transormationsverhalten

### Package Import
importieren des Packages Numpy

In [ ]:
import numpy as np

### Eingabe der Werte

In [ ]:
E_11    =   42.3E3  # MPa
E_22    =   11.5E3  # MPa
G_12    =   4.5E3   # MPa
nu_12   =   0.290
nu_21   =   0.074

alpha = np.radians(45)  # important to set angle in radians!

### Spannungstensor
Eingabe des Spannungstenors in pseudo-vektorieller Darstellung (Voigt-Notation)

$$\underline{\sigma} = [\sigma_x,\sigma_y,\tau_{xy}]^{T} = [0,50,0]^{T}\; \text{MPa}$$

Eingabe als Numpy-Array

In [ ]:
stress_xy = np.array([0,50,0])

### Berechnung der Nachgiebigkeitsmatrix
#### Material Koodrinatensystem ($\parallel,\bot$ bzw $1,2$)

$$[S_{ij}] = \begin{bmatrix} S_{11} & S_{12} & S_{16}\\
S_{21} & S_{22} & S_{26}\\
S_{61} & S_{62} & S_{66} \end{bmatrix}$$

für transversale Isotropie

$$[S_{ij}]_{(\parallel,\bot)} = \begin{bmatrix} S_{11} & S_{12} & S_{16}\\
S_{21} & S_{22} & S_{26}\\
S_{61} & S_{62} & S_{66} \end{bmatrix}=\begin{bmatrix} \frac{1}{E_{\parallel}} & -\frac{\nu_{\parallel\bot}}{E_\parallel} & 0\\
-\frac{\nu_{\parallel\bot}}{E_\parallel} & \frac{1}{E_{\bot}} & 0\\
0 & 0 & \frac{1}{G_{\parallel\bot}} \end{bmatrix}$$


In [ ]:
S11 =   1/E_11
S22 =   1/E_22  
S66 =   1/G_12
S12 =   -nu_12/E_11
S16 =   0
S26 =   0
S61 =   S16 # symmetry
S62 =   S26 # symmetry

In [ ]:
S_12 = np.array([[S11,S12,S16],[S12,S22,S26],[S61,S62,S66]])
print("Compliance Matrix S_ij\n",S_12)

### Transformation

$$ [T]^{(\varepsilon)}_{\parallel\bot \rightarrow x,y}=
\begin{pmatrix} 
\cos^2\alpha & \sin^2\alpha & -0.5\sin{2\alpha} \\
\sin^2\alpha & \cos^2\alpha & 0.5\sin{2\alpha} \\
\sin{2\alpha} & -\sin{2\alpha} & \cos{2\alpha}
\end{pmatrix}$$

Hinweis:

$$ \sin{2\alpha} = 2\cos\alpha\sin\alpha$$

$$ \cos{2\alpha} = \cos^2\alpha - \sin^2\alpha$$

Beziehungen zwischen den Transformationsmatrizen

für die Transformation zwischen den Koordinatensystemen:
$$ [T]^{(\varepsilon)}_{x,y \rightarrow \parallel\bot} = \left\{ [T]^{(\varepsilon)}_{\parallel\bot \rightarrow x,y} \right\}^{-1}$$

für die Spannungstransformationen:
$$ [T]^{(\sigma)}_{ \parallel\bot \rightarrow x,y  } = \left\{\left\{ [T]^{(\varepsilon)}_{\parallel\bot \rightarrow x,y} \right\}^{T}\right\}^{-1}$$

$$ [T]^{(\sigma)}_{  x,y \rightarrow\parallel\bot  } = \left\{ [T]^{(\varepsilon)}_{\parallel\bot \rightarrow x,y} \right\}^{T}$$

Im nachfolgenden gilt:

$$ [T] = [T]^{(\varepsilon)}_{\parallel\bot \rightarrow x,y}=
\begin{pmatrix} 
\cos^2\alpha & \sin^2\alpha & -0.5\sin{2\alpha} \\
\sin^2\alpha & \cos^2\alpha & 0.5\sin{2\alpha} \\
\sin{2\alpha} & -\sin{2\alpha} & \cos{2\alpha}
\end{pmatrix}$$


### Erstellung der Transformationsmatrix $[T]$ für gegebenen Winkel $\alpha$

In [ ]:
## T represent the transformation matrix for T_(1,2->x,y) for strain
T = np.array([
    [np.cos(alpha)**2, np.sin(alpha)**2, -0.5 * np.sin(2*alpha)],
    [np.sin(alpha)**2, np.cos(alpha)**2, 0.5 * np.sin(2*alpha)],
    [np.sin(2*alpha), -np.sin(2*alpha), np.cos(2*alpha)]
])

print("Transformation Matrix T:")
print(T)

### Berechnung der Nachgeibigkeitsmatrix $[S]_{(x,y)}$ im (x,y)-Koordinatensystem

$$[S]_{(x,y)} = [T]^{(\varepsilon)}_{\parallel\bot \rightarrow x,y} [S]_{(\parallel,\bot)} [T]^{(\sigma)}_{x,y \rightarrow \parallel\bot  } = [T] [S]_{(\parallel,\bot)} \left\{ [T] \right\}^{T}$$

#### Funktion `np.matmul()`

Die in `numpy` implementierte Funktion `np.matmul()` is als "Matrix product of two arrays" definiert. Weiterführende Information [hier](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html)


In [ ]:
S_xy = np.matmul(np.matmul(T, S_12), np.transpose(T))

print("Nachgiebigkeits Matrix S_xy (xy)-Koordinatensystem:")
print(S_xy)

### Berechnung des Dehnungstensors $\underline{\varepsilon}_{(x,y)}$

Verwendung der Spannungs-Dehnungs-Beziehung des Hooke'schen Gesetzes

$$\underline{\varepsilon}_{(x,y)} = [S]_{(x,y)} \underline{\sigma}_{(x,y)}$$

In [ ]:
strain_xy = np.matmul(S_xy,stress_xy)

print("Komponenten des Dehnungstensors strain_xy:\n",strain_xy)

### Berechnung des Dehnungstensors $\underline{\varepsilon}_{(\parallel,\bot)}$

In [ ]:
strain_12 = np.matmul(np.linalg.inv(T),strain_xy)
print(strain_12)

### Berechnung der Spannungen im Material Koordiantensystem ($\parallel,\bot$ bzw $1,2$)

#### Variante 1: Spannungsdehnungsbeziehung

$$\underline{\sigma}_{(\parallel,\bot)} = \left\{ [S]_{(\parallel,\bot)} \right\}^{-1} \underline{\varepsilon}_{(\parallel,\bot)}$$

In [ ]:
stress_12 = np.matmul(np.linalg.inv(S_12),strain_12)
print(stress_12)

#### Variante 2: Transforamtionsbeziehungen

In [ ]:
stress_12 = np.matmul(np.transpose(T),stress_xy)

print(stress_12)

#### Variante 3: Moohr'scher Spannungskreis

siehe Lösungen zu der Aufgabe UE6.4

### Berechnung des Versagenskriteriums nach TsaiWu

Die allgeimeine Darstellung des TsaiWu-Versagenskriterium für den ebenen Spannungszustand ist wie folgt:
$$ \sum_{i,j=1,2,6} F_{i}\sigma_i + F_{ij} \sigma_{i}\sigma_{j}\leq 1$$
Dabei sind die Spannungen $\sigma_i$ für das $(\parallel,\bot)$-(Material)-Koordinatensystem wie folgt definiert (Voigt-Notation; pseudo-vekotrielle Darstellung):

$$\underline{\sigma}_{(\parallel,\bot)}= \left[\sigma_\parallel, \sigma_\bot, \tau_{\parallel,\bot} \right]=\left[\sigma_1, \sigma_2, \sigma_6 \right]$$

Daraus folgt die folgende Definition des TsaiWu Versagenskriteriums
$$F_{1} \sigma_\parallel + F_{11} \sigma_\parallel^2 +F_{12}\sigma_{\parallel\bot} F_{2} \sigma_\bot + F_{2} \sigma_\bot^2 + F_{66}\tau_{\parallel\bot}\leq 1$$

Hinweis: 

- lineare Schubspannungsglieder sind per Definition zu vernachlässigen
- $F_{12}=0$ (konservative Auslegung)




#### Eingabe der Basisfestigkeiten der UD-Schicht

In [ ]:
R11t    =   650 #MPa
R11c    =   600 #MPa
R22t    =   65  #MPa
R22c    =   120 #MPa
R12     =   38  #MPa

#### Berechnung des Festigkeitskoefizienten $F_{i}$ und $F_{ij}$

$$F_{1} = \frac{1}{R_1^+}- \frac{1}{R_1^-} \qquad F_{2} = \frac{1}{R_2^+}- \frac{1}{R_2^-}$$

$$F_{11} = \frac{1}{R_1^+R_1^-}\qquad F_{22} = \frac{1}{R_2^+R_2^-}\qquad F_{66} = \frac{1}{\left(R_{12}\right)^2}$$



In [ ]:
F1 = 1/R11t-1/R11c
F2 = 1/R22t-1/R22c

F11 =   1/(R11t*R11c)
F22 =   1/(R22t*R22c)
F66 =   1/(R12)**2
F12 =   0

In [ ]:
print("F1: %.3E; F2: %.3E;F11: %.3E; F22: %.3E; F66: %.3E; F12: %.3E" %(F1,F2,F11,F22,F66,F12))

In [ ]:
def tsaiwu2D(stress,coeff):
    """
    input:  
        -   stress: 3x1 numpy array with stress components 
        -   coeff: 6x1 numpy array with the strength coeff F1,F2,F11,F22,F66,F12
    output: 
        -   value: value of TsaiWu criteria
        -   fres: reverse faktor of TsaiWu criteria
    """
    F1 = coeff[0]
    F2 = coeff[1]
    F11 =   coeff[2]
    F22 =   coeff[3]
    F66 =   coeff[4]
    F12 =   coeff[5]

    # calcualte the value of TsaiWu criteria
    value = F1*stress[0]+ F11*stress[0]**2 +F2*stress[1]+ F22*stress[1]**2 + F66*stress[2]**2

    # calcualte the reserve factor of TsaiWu criteria
    a = F11*stress[0]**2 + F12*stress[0]*stress[1] + F22*stress[1]**2+F66*stress[2]**2
    b = F1*stress[0] +F2*stress[1]
    c = -1
    print(a,b)
    # function np.roots return the roots of a polynomial with coefficients given in p
    # https://numpy.org/doc/stable/reference/generated/numpy.roots.html
    fres = np.roots([a, b, c])[1] 
    
    # return all values
    return value, fres

In [ ]:
value, fres = tsaiwu2D([25,25,25],[F1,F2,F11,F22,F66,F12])

print("The value of TsaiWu cirteria: %.3E\nand the reserve factor fres: %.3E" %(value,fres))